In [1]:
import pandas as pd
import json

tracks_df = pd.read_json('./../data/raw/05_02_v2/tracks.jsonl', lines=True)
users_df = pd.read_json('./../data/raw/05_02_v2/users.jsonl', lines=True)
session_df = pd.read_json('./../data/raw/05_02_v2/sessions.jsonl', lines=True)
artists = pd.read_json('./../data/raw/05_02_v2/artists.jsonl', lines=True)

In [2]:
for x in [tracks_df, users_df, session_df, artists]:
    print(x.isnull().sum(), x.shape, '\n')

id                  0
name                0
popularity          0
duration_ms         0
explicit            0
id_artist           0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64 (22412, 17) 

user_id             0
name                0
city                0
street              0
favourite_genres    0
premium_user        0
dtype: int64 (500, 6) 

session_id         0
timestamp          0
user_id            0
track_id      102466
event_type         0
dtype: int64 (759425, 5) 

id        0
name      0
genres    0
dtype: int64 (1667, 3) 



In [3]:
session_df[session_df['track_id'].isna() & (session_df['event_type'] != 'advertisment')].shape

(0, 5)

## Combine artist with tracks to get track genre

In [32]:
tracks_df.columns, artists.columns
tracks_genre_df = tracks_df.merge(artists[['id', 'genres']], left_on='id_artist', right_on='id', how='inner')
tracks_genre_df

,id_x,name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id_y,genres
0,0RNxWy0PC3AyH4ThH3aGK6,Mack the Knife,55,201467,0,19eLuQmk9aCobbVDHc6eek,1929,0.673,0.3770,0,-14.141,0.0697,0.586000,0.000000,0.332,0.713,88.973,19eLuQmk9aCobbVDHc6eek,"[adult standards, dixieland, harlem renaissanc..."
1,17gxfuiFUrLhbUKdunxUPJ,Moon River,58,179867,0,19eLuQmk9aCobbVDHc6eek,1964-10-25,0.448,0.1200,0,-14.089,0.0355,0.877000,0.013500,0.100,0.261,86.407,19eLuQmk9aCobbVDHc6eek,"[adult standards, dixieland, harlem renaissanc..."
2,63kd4m3VFxcJjPVVtbVNAu,"Hello, Dolly!",53,147000,0,19eLuQmk9aCobbVDHc6eek,1964-10-25,0.000,0.4050,0,-9.935,0.0000,0.842000,0.001140,0.198,0.000,0.000,19eLuQmk9aCobbVDHc6eek,"[adult standards, dixieland, harlem renaissanc..."
3,1qCQTy0fTXerET4x8VHyr9,What A Wonderful World,74,137520,0,19eLuQmk9aCobbVDHc6eek,1968,0.399,0.2580,5,-16.028,0.0330,0.792000,0.000002,0.128,0.192,108.174,19eLuQmk9aCobbVDHc6eek,"[adult standards, dixieland, harlem renaissanc..."
4,1UH4viviUjZnS9aWgPGrk0,La vie en rose - Single Version,66,204400,0,19eLuQmk9aCobbVDHc6eek,1989-01-01,0.507,0.0779,0,-12.099,0.0544,0.866000,0.002750,0.108,0.326,70.808,19eLuQmk9aCobbVDHc6eek,"[adult standards, dixieland, harlem renaissanc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22407,7yvgurK6MRYwTbZ2UOgkgj,Jesteśmy tu,56,187200,1,4nPxrGG7k7aEKmNLsfX4cd,2020-08-28,0.779,0.6140,11,-6.631,0.1090,0.496000,0.000000,0.131,0.554,99.919,4nPxrGG7k7aEKmNLsfX4cd,"[polish hip hop, polish trap]"
22408,62Qu77wJcmICTHse0f9qsQ,Wschód (Intro),53,145213,1,4nPxrGG7k7aEKmNLsfX4cd,2020-08-28,0.741,0.5880,2,-7.531,0.1300,0.474000,0.000004,0.149,0.798,96.012,4nPxrGG7k7aEKmNLsfX4cd,"[polish hip hop, polish trap]"
22409,1Ddj6nkhUP3lHpLvDqPsvC,Strip That Down - Acoustic,57,184476,0,5pUo3fmmHT8bhCyHE52hA6,2017-06-16,0.880,0.3800,6,-7.811,0.0605,0.639000,0.000000,0.124,0.912,105.982,5pUo3fmmHT8bhCyHE52hA6,"[dance pop, pop, post-teen pop]"
22410,1Ca2ELRlvACAeI7xz9c9jR,Spaceman,54,379261,0,6BrvowZBreEkXzJQMpL174,2012-02-06,0.599,0.7630,3,-3.727,0.0553,0.000262,0.794000,0.241,0.118,127.997,6BrvowZBreEkXzJQMpL174,"[big room, dance pop, dutch house, edm, electr..."


In [54]:
track_genre_pairs = []
for idx, row in tracks_genre_df[['id_x', 'genres']].iterrows():
    track_id, genres = row['id_x'], row['genres']

    # For each genre, create a pair with the track_id
    for genre in genres:
        track_genre_pairs.append({'track_id': track_id, 'genre': genre})

# Create a dataframe from the list of dictionaries
tracks_expanded_genres_df = pd.DataFrame(track_genre_pairs, columns=['track_id', 'genre'])
print(tracks_expanded_genres_df)


# with open('../data/processed/track_genres.json', 'w') as f:
tracks_expanded_genres_df.to_json('../data/processed/track_genres.json')

                     track_id                      genre
0      0RNxWy0PC3AyH4ThH3aGK6            adult standards
1      0RNxWy0PC3AyH4ThH3aGK6                  dixieland
2      0RNxWy0PC3AyH4ThH3aGK6         harlem renaissance
3      0RNxWy0PC3AyH4ThH3aGK6               jazz trumpet
4      0RNxWy0PC3AyH4ThH3aGK6           new orleans jazz
...                       ...                        ...
90684  1Ca2ELRlvACAeI7xz9c9jR                  pop dance
90685  1Ca2ELRlvACAeI7xz9c9jR  progressive electro house
90686  1Ca2ELRlvACAeI7xz9c9jR          progressive house
90687  1Ca2ELRlvACAeI7xz9c9jR             tropical house
90688  0XwXxgC5qyCF6PoyAREAzK                pet calming

[90689 rows x 2 columns]


In [57]:
genre_counts = tracks_expanded_genres_df.groupby('genre')['track_id'].count()
print(genre_counts)

for genre, group in tracks_expanded_genres_df.groupby('genre'):
    print(f"Genre: {genre}")
    print(group.head())  # Print first few rows of each group

genre
a cappella           3
acid rock           57
acoustic pop        30
adult standards    510
afro dancehall       8
                  ... 
worship             65
wrestling           68
yacht rock         216
zhongguo feng       43
zolo                18
Name: track_id, Length: 779, dtype: int64
Genre: a cappella
                     track_id       genre
77431  550rQQCGkrTzvp4SfpOPzx  a cappella
77432  6ukvsBzq4d1vBsAUmz7ZVt  a cappella
77433  1klGbW5a9qTBFUjFfddbmU  a cappella
Genre: acid rock
                    track_id      genre
3938  5uvosCdMlFdTXhoazkTI5R  acid rock
3943  6ToM0uwxtPKo9CMpbPGYvM  acid rock
3948  1Jmqubf9kGkWeYQXQKImL5  acid rock
3953  67HxeUADW4H3ERfaPW59ma  acid rock
3958  2Xdc6qyaFBJZ8QW1KhpVci  acid rock
Genre: acoustic pop
                     track_id         genre
46136  4tK4ARuilOrBzwVQK73E0Y  acoustic pop
46141  2mKjs6s0Z1imKKb6gOk628  acoustic pop
46146  1EzrEOXmMH3G43AXT1y7pA  acoustic pop
46151  3S0OXQeoh0w6AY8WQVckRW  acoustic pop
46156  5ivF4eQBq

In [8]:
# tracks_genres
# tracks_genres_df = pd.read_json('../data/processed/track_genres.json')

KeyboardInterrupt: 